# K-겹 교차 검증 (K-Fold Cross-Validation): 모델 성능의 신뢰성 확보

이 노트북은 머신러닝 모델의 성능을 보다 신뢰성 있게 평가하기 위한 핵심 기법인 **K-겹 교차 검증(K-Fold Cross-Validation)**을 다룹니다. 특히, 일반적인 `KFold`와 분류 문제에 더 적합한 `StratifiedKFold`의 차이점을 이해하고, `scikit-learn`의 `cross_val_score` 함수를 사용하여 교차 검증을 간편하게 수행하는 방법을 알아봅니다.

### 교차 검증의 필요성
모델을 훈련 세트와 테스트 세트로 한 번만 나누어 평가할 경우, 데이터 분할 방식에 따라 모델의 성능 평가가 불안정하거나 과대적합(Overfitting)될 위험이 있습니다. 교차 검증은 이러한 문제를 해결하고 모델의 일반화 성능을 더 정확하게 추정하는 데 도움을 줍니다.

### 1. 라이브러리 임포트 및 데이터 로드

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier

# 붓꽃 데이터셋 로드
iris = load_iris()
X = iris.data 
y = iris.target 

print(f"데이터 형태: {X.shape}")
print(f"타겟 형태: {y.shape}")
print(f"클래스 분포: {np.bincount(y)}") # 0, 1, 2 클래스의 샘플 개수 확인

### 2. K-Fold 교차 검증

`KFold`는 데이터를 K개의 동일한 크기의 '폴드(fold)'로 나눕니다. 각 반복마다 하나의 폴드를 테스트 세트로 사용하고, 나머지 K-1개의 폴드를 훈련 세트로 사용합니다. 이 과정을 K번 반복하여 K개의 성능 점수를 얻고, 이들의 평균을 최종 성능으로 간주합니다.

In [ ]:
train_scores_kfold = []
test_scores_kfold = []

# n_splits: 폴드의 개수 (여기서는 5개로 나눔)
kfold = KFold(n_splits=5, shuffle=True, random_state=42) # shuffle과 random_state로 재현성 확보

print("--- K-Fold 교차 검증 시작 ---")
for fold, (train_index, test_index) in enumerate(kfold.split(X)):
    # 각 폴드에 해당하는 데이터 분리
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # 모델 학습 및 평가
    model = DecisionTreeClassifier(random_state=42) # 재현성을 위해 random_state 설정
    model.fit(X_train, y_train)
    
    train_scores_kfold.append(model.score(X_train, y_train))
    test_scores_kfold.append(model.score(X_test, y_test))
    print(f"Fold {fold+1}: 훈련 점수 = {train_scores_kfold[-1]:.4f}, 테스트 점수 = {test_scores_kfold[-1]:.4f}")

print("
K-Fold 훈련 점수들:", train_scores_kfold)
print("K-Fold 테스트 점수들:", test_scores_kfold)
print(f"K-Fold 테스트 점수 평균: {np.mean(test_scores_kfold):.4f}")

print("
**K-Fold의 한계**: 데이터셋에 클래스 불균형이 있을 경우, 특정 폴드에 특정 클래스의 샘플이 몰리거나 아예 없을 수 있어 성능 평가가 왜곡될 수 있습니다.")

### 3. 계층별 K-Fold 교차 검증 (Stratified K-Fold)

`StratifiedKFold`는 `KFold`의 단점을 보완한 방법으로, 각 폴드에 원본 데이터셋의 클래스 비율을 유지하면서 데이터를 분할합니다. 이는 분류 문제, 특히 클래스 불균형이 있는 데이터셋에서 모델 성능을 더 정확하게 평가하는 데 필수적입니다.

In [ ]:
train_scores_stratified = []
test_scores_stratified = []

# n_splits: 폴드의 개수
# shuffle: 데이터를 섞을지 여부
# random_state: 재현성을 위한 시드
sfk = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) 

print("--- Stratified K-Fold 교차 검증 시작 ---")
# split 함수에 X와 y를 모두 전달하여 클래스 비율을 고려하도록 합니다.
for fold, (train_index, test_index) in enumerate(sfk.split(X, y)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    train_scores_stratified.append(model.score(X_train, y_train))
    test_scores_stratified.append(model.score(X_test, y_test))
    print(f"Fold {fold+1}: 훈련 점수 = {train_scores_stratified[-1]:.4f}, 테스트 점수 = {test_scores_stratified[-1]:.4f}")

print("
Stratified K-Fold 훈련 점수들:", train_scores_stratified)
print("Stratified K-Fold 테스트 점수들:", test_scores_stratified)
print(f"Stratified K-Fold 테스트 점수 평균: {np.mean(test_scores_stratified):.4f}")

print("**결과**: Stratified K-Fold는 각 폴드에서 클래스 비율이 균등하게 유지되므로, K-Fold보다 더 안정적이고 신뢰할 수 있는 성능 평가를 제공합니다.")

### 4. `cross_val_score`를 이용한 간편한 교차 검증

`scikit-learn`의 `cross_val_score` 함수는 위에서 수동으로 구현한 교차 검증 과정을 한 줄로 간편하게 수행할 수 있도록 해줍니다.

- `estimator`: 학습할 모델 객체
- `X`, `y`: 전체 데이터
- `scoring`: 평가 지표 (예: 'accuracy', 'f1', 'roc_auc')
- `cv`: 교차 검증 폴드의 개수 또는 교차 검증 전략 객체 (예: `KFold`, `StratifiedKFold` 객체)

In [ ]:
model_cv = DecisionTreeClassifier(random_state=42)

# cv=5는 기본적으로 StratifiedKFold (분류 문제의 경우)를 사용합니다.
scores = cross_val_score(model_cv, X, y, scoring="accuracy", cv=5 ) 

print("--- cross_val_score 결과 ---")
print("각 폴드별 정확도:", scores)
print(f"평균 정확도: {np.mean(scores):.4f}")
print(f"정확도 표준편차: {np.std(scores):.4f}")

### 결론

교차 검증은 모델의 일반화 성능을 신뢰성 있게 평가하고, 과대적합을 방지하는 데 필수적인 기법입니다. 특히 `StratifiedKFold`는 분류 문제에서 클래스 불균형을 고려하여 더 안정적인 평가를 제공합니다. `cross_val_score`와 같은 편리한 함수를 활용하여 효율적으로 교차 검증을 수행할 수 있습니다.